In [11]:
import os
import pandas as pd
import json
import traceback
from dotenv import load_dotenv
load_dotenv() #take environment variables from .env.

True

In [13]:
from langchain.chat_models import ChatOpenAI
KEY = os.getenv("apikey")


In [15]:
llm=ChatOpenAI (openai_api_key=KEY, model_name="gpt-3.5-turbo", temperature=0.5)

e:\ALL PROGRAMS\All_DS_Projects\Generative AI\MCQ_Gen\mcqenv\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(
